In [36]:
from graphdatascience import GraphDataScience
import pandas as pd
from py2neo import Node, Graph, Relationship,NodeMatcher
import math
from neo4j import GraphDatabase

In [37]:
df = pd.read_csv("../FinalDataSetCreation/FinalDataset_with_all.csv")
if 'Unnamed: 0' in df.columns:
    df.drop(['Unnamed: 0'], axis = 1, inplace= True)
work_id_list = list(df['id'])
recommend_dict = {}
for work_id in work_id_list: recommend_dict[work_id] = []

query_list= []
for work_id in work_id_list:
    query = "MATCH (n:Artork)-[r:SIMILAR]->(m:Artork) WHERE n.id = '{}' RETURN m.id as id, r.score as similarity ORDER BY similarity DESCENDING LIMIT 8".format(work_id)
    recommend_dict[work_id].append(query)

In [38]:
dummy = """MATCH (n:Artork)-[r:SIMILAR]->(m:Artork)
WHERE n.id = 'prints89'
RETURN m.id as id, r.score as similarity
ORDER BY similarity DESCENDING LIMIT 8"""


In [39]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "myneo4j"))

with driver.session() as session:
    for work_id in recommend_dict:
        results = session.run(recommend_dict[work_id][0]).data()
        for i in range(8):
            recommend_dict[work_id].append(results[i]['id'])
        recommend_dict[work_id] = recommend_dict[work_id][1:]
        print('done {}'.format(work_id))


done painting15
done prints369
done prints826
done photography133
done painting37
done painting52
done painting58
done painting63
done painting83
done painting104
done painting168
done painting300
done painting762
done prints1076
done painting41
done prints23
done prints566
done works-on-paper44
done painting45
done painting172
done works-on-paper1197
done mixed-media1254
done painting65
done painting86
done painting91
done painting100
done painting208
done prints89
done prints218
done prints264
done prints290
done prints312
done prints325
done prints381
done prints385
done prints541
done prints811
done sculpture46
done sculpture1110
done works-on-paper373
done works-on-paper500
done works-on-paper721
done works-on-paper779
done works-on-paper1041
done works-on-paper1059
done mixed-media627
done mixed-media638
done mixed-media640
done mixed-media670
done mixed-media673
done mixed-media723
done mixed-media736
done mixed-media761
done mixed-media926
done mixed-media1000
done mixed-media1

In [34]:
sim = 'similar'
colist = []
for i in range(1,9):
    colist.append(sim+str(i))


In [35]:
rec_df = pd.DataFrame.from_dict(recommend_dict, orient='index', columns = colist)
rec_df.reset_index(inplace=True)
rec_df.columns = ['id']+colist
rec_df

,id,similar1,similar2,similar3,similar4,similar5,similar6,similar7,similar8
0,painting15,prints1420,prints826,photography377,sculpture706,prints1385,prints1282,design7,prints408
1,prints369,painting882,photography391,photography575,prints429,photography190,sculpture1049,sculpture990,painting104
2,prints826,prints1420,works-on-paper250,painting15,sculpture370,works-on-paper415,prints1084,prints948,works-on-paper1246
3,photography133,sculpture173,design179,design37,sculpture485,sculpture1218,photography293,photography933,prints969
4,painting37,prints596,prints1373,works-on-paper1002,prints1076,mixed-media1212,prints218,prints587,sculpture128
...,...,...,...,...,...,...,...,...,...
2474,design1182,sculpture190,works-on-paper740,prints1309,works-on-paper679,sculpture264,painting1245,mixed-media1476,prints555
2475,design1269,works-on-paper1453,painting1024,prints468,works-on-paper44,works-on-paper1331,prints1021,prints4,photography882
2476,design1278,prints1147,mixed-media1379,prints659,prints1438,prints1406,prints284,prints823,design638
2477,design1440,mixed-media1093,mixed-media638,mixed-media1235,photography1296,works-on-paper424,mixed-media950,prints715,mixed-media919


In [44]:
df = df.merge(rec_df, how = 'left', on = 'id')
df.to_csv("../FinalDataSetCreation/FinalDataset_with_recommendation.csv")
